In [9]:
import os
import pandas as pd
from datetime import datetime, timezone
import pytz
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Process Reid's Listening History

In [93]:
def clean_json(json):
    json = json.drop(["username", "ip_addr_decrypted", "user_agent_decrypted", "episode_name", "episode_show_name", "spotify_episode_uri", "offline_timestamp"], axis=1)
    json = json.rename({"master_metadata_track_name": "track", "master_metadata_album_artist_name": "artist", "master_metadata_album_album_name": "album"}, axis=1)
    return json

In [94]:
path = "reid_data"
files = [file for file in os.listdir(path) if file.startswith('Streaming_History_Audio')]
combined = pd.DataFrame()

for file in files:
    current_file = clean_json(pd.read_json(path + "/" + file))
    combined = pd.concat([combined, current_file])

In [95]:
combined = combined.drop(combined[(combined["artist"] == "Milli Hughes") | (combined["artist"] == "The Noisy Outdoors") | (combined["artist"] == "Atmosphere of Calm")].index).reset_index()
combined["ts"] = [datetime.strptime(i, "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=timezone.utc).astimezone(pytz.timezone('US/Eastern')) for i in combined["ts"]]
combined["spotify_track_uri"] = combined["spotify_track_uri"].map(lambda x: x and x.lstrip('spotify:track:'))
combined = combined[~combined["spotify_track_uri"].isna()] # removes about 100 rows with a missing track uri
combined

,index,ts,platform,ms_played,conn_country,track,artist,album,spotify_track_uri,reason_start,reason_end,shuffle,skipped,offline,incognito_mode
0,0,2023-01-30 16:17:04-05:00,osx,234146,US,Miss Americana & The Heartbreak Prince,Taylor Swift,Lover,214nt20w5wOxJnY462klLw,trackdone,trackdone,True,False,False,False
1,1,2023-01-30 16:21:04-05:00,osx,239013,US,Cold As You,Taylor Swift,Taylor Swift,7an1exwMnfYRcdVQm0yDev,trackdone,trackdone,True,False,False,False
2,2,2023-01-30 16:24:44-05:00,osx,220040,US,hoax,Taylor Swift,folklore,0YeDG5HnKnG7jpArkzsSPa,trackdone,trackdone,True,False,False,False
3,3,2023-01-30 16:28:45-05:00,osx,240466,US,The Lucky One (Taylor's Version),Taylor Swift,Red (Taylor's Version),4e5ayHsOLJNLTGfjau2mEw,trackdone,trackdone,True,False,False,False
4,4,2023-01-30 16:32:17-05:00,osx,211506,US,Dancing With Our Hands Tied,Taylor Swift,reputation,7I7JbDv63ZJJsSi24DyJrz,trackdone,trackdone,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32179,16301,2023-01-30 16:00:31-05:00,osx,231000,US,the last great american dynasty,Taylor Swift,folklore,2olxzvoFI9IpxqFeUv7WOX,trackdone,trackdone,True,0.0,False,False
32180,16302,2023-01-30 16:00:38-05:00,osx,6778,US,Blank Space - Voice Memo,Taylor Swift,1989,0yisH8r0f3Mkq34kjzUlCV,trackdone,fwdbtn,True,1.0,False,False
32181,16303,2023-01-30 16:04:35-05:00,osx,237338,US,Come In With The Rain (Taylor’s Version),Taylor Swift,Fearless (Taylor's Version),1n2wszmJyVkw6FHqyLnQsY,fwdbtn,trackdone,True,0.0,False,False
32182,16304,2023-01-30 16:08:56-05:00,osx,260361,US,"Would've, Could've, Should've",Taylor Swift,Midnights,0aV5uARAknQgYhBaK944FP,trackdone,trackdone,True,0.0,False,False


# Query Song Information from Spotify's API for Each Unique Track in Reid's Listening History

In [54]:
unique_track_uris = combined["spotify_track_uri"].unique()
print(len(unique_track_uris))
unique_track_uris

5562


array(['214nt20w5wOxJnY462klLw', '7an1exwMnfYRcdVQm0yDev',
       '0YeDG5HnKnG7jpArkzsSPa', ..., '3oJtUnnt5uYPtzulIbLw3D',
       '4TMVC63HZDYaTGtlhbM95i', '1XpYodsD36XN7ygcdF7mJJ'], dtype=object)

In [56]:
auth_manager = SpotifyClientCredentials(client_id="38d867f8039e43fd9f03ac9c65eabcac", client_secret="2059d30cfe234742bb000fd2777564da")
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
for i in range(0, len(unique_track_uris), 100):
    print("starting", i, "to", i+100)
    query = sp.audio_features(unique_track_uris[i:i+100])
    tmp = pd.DataFrame(song for song in query if song is not None) # removed about 10 songs which did not return any data from the API
    if i == 0:
        track_aud_feats = tmp
    else:
        track_aud_feats = pd.concat([track_aud_feats, tmp])
    print("finished", i, "to", i+100)

In [82]:
track_aud_feats.to_csv("reid_track_audio_features.csv")

# Combine the Two Datasets

In [96]:
combined = combined.set_index("spotify_track_uri").drop("index", axis=1)
combined

,ts,platform,ms_played,conn_country,track,artist,album,reason_start,reason_end,shuffle,skipped,offline,incognito_mode
spotify_track_uri,,,,,,,,,,,,,
214nt20w5wOxJnY462klLw,2023-01-30 16:17:04-05:00,osx,234146,US,Miss Americana & The Heartbreak Prince,Taylor Swift,Lover,trackdone,trackdone,True,False,False,False
7an1exwMnfYRcdVQm0yDev,2023-01-30 16:21:04-05:00,osx,239013,US,Cold As You,Taylor Swift,Taylor Swift,trackdone,trackdone,True,False,False,False
0YeDG5HnKnG7jpArkzsSPa,2023-01-30 16:24:44-05:00,osx,220040,US,hoax,Taylor Swift,folklore,trackdone,trackdone,True,False,False,False
4e5ayHsOLJNLTGfjau2mEw,2023-01-30 16:28:45-05:00,osx,240466,US,The Lucky One (Taylor's Version),Taylor Swift,Red (Taylor's Version),trackdone,trackdone,True,False,False,False
7I7JbDv63ZJJsSi24DyJrz,2023-01-30 16:32:17-05:00,osx,211506,US,Dancing With Our Hands Tied,Taylor Swift,reputation,trackdone,trackdone,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2olxzvoFI9IpxqFeUv7WOX,2023-01-30 16:00:31-05:00,osx,231000,US,the last great american dynasty,Taylor Swift,folklore,trackdone,trackdone,True,0.0,False,False
0yisH8r0f3Mkq34kjzUlCV,2023-01-30 16:00:38-05:00,osx,6778,US,Blank Space - Voice Memo,Taylor Swift,1989,trackdone,fwdbtn,True,1.0,False,False
1n2wszmJyVkw6FHqyLnQsY,2023-01-30 16:04:35-05:00,osx,237338,US,Come In With The Rain (Taylor’s Version),Taylor Swift,Fearless (Taylor's Version),fwdbtn,trackdone,True,0.0,False,False


In [87]:
track_aud_feats = pd.read_csv("reid_track_audio_features.csv")
track_aud_feats = track_aud_feats.set_index("id").drop(["Unnamed: 0", "type", "uri", "track_href", "analysis_url"], axis=1)
track_aud_feats

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
id,,,,,,,,,,,,,
214nt20w5wOxJnY462klLw,0.662,0.747,11,-6.926,0,0.0736,0.02800,0.006150,0.1380,0.487,150.088,234147,4
7an1exwMnfYRcdVQm0yDev,0.418,0.482,5,-5.769,1,0.0266,0.21700,0.000000,0.1230,0.261,175.558,239013,4
0YeDG5HnKnG7jpArkzsSPa,0.676,0.178,8,-15.010,1,0.0394,0.96400,0.000008,0.1350,0.404,118.840,220040,4
4e5ayHsOLJNLTGfjau2mEw,0.686,0.571,4,-7.138,1,0.0500,0.06600,0.000000,0.0608,0.538,117.889,240467,4
7I7JbDv63ZJJsSi24DyJrz,0.624,0.691,0,-6.686,0,0.1960,0.06040,0.000011,0.1380,0.284,160.024,211507,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4Wd9pEtEnZNDjgiswGOpJb,0.611,0.794,0,-6.042,1,0.0283,0.15500,0.036500,0.5680,0.905,116.981,274227,4
4tXr9v3K7nWW1aebWqR1B7,0.603,0.742,4,-4.363,0,0.0327,0.00286,0.001050,0.2560,0.470,105.036,271160,4
3oJtUnnt5uYPtzulIbLw3D,0.540,0.863,2,-4.599,1,0.0284,0.33000,0.000018,0.1560,0.931,130.354,244533,4


In [ ]:
final_df = pd.merge(left=combined, right=track_aud_feats, how="inner", left_index=True, right_index=True)
final_df

,ts,platform,ms_played,conn_country,track,artist,album,reason_start,reason_end,shuffle,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
003vvx7Niy0yvhvHt4a68B,2023-03-09 15:37:04-05:00,osx,25216,US,Mr. Brightside,The Killers,Hot Fuss,clickrow,endplay,False,...,-5.230,1,0.0747,0.00121,0.000000,0.0995,0.236,148.033,222973,4
003vvx7Niy0yvhvHt4a68B,2023-04-03 18:34:06-04:00,ios,222973,US,Mr. Brightside,The Killers,Hot Fuss,trackdone,trackdone,True,...,-5.230,1,0.0747,0.00121,0.000000,0.0995,0.236,148.033,222973,4
003vvx7Niy0yvhvHt4a68B,2023-05-07 13:05:06-04:00,osx,222973,US,Mr. Brightside,The Killers,Hot Fuss,trackdone,trackdone,True,...,-5.230,1,0.0747,0.00121,0.000000,0.0995,0.236,148.033,222973,4
003vvx7Niy0yvhvHt4a68B,2023-07-05 02:27:51-04:00,osx,222973,US,Mr. Brightside,The Killers,Hot Fuss,trackdone,trackdone,False,...,-5.230,1,0.0747,0.00121,0.000000,0.0995,0.236,148.033,222973,4
003vvx7Niy0yvhvHt4a68B,2023-07-18 15:46:20-04:00,osx,12330,IL,Mr. Brightside,The Killers,Hot Fuss,clickrow,endplay,True,...,-5.230,1,0.0747,0.00121,0.000000,0.0995,0.236,148.033,222973,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7zsXy7vlHdItvUSH8EwQss,2022-05-21 19:40:44-04:00,"iOS 15.4.1 (iPhone9,1)",32531,US,Cold Water,Major Lazer,Cold Water,fwdbtn,fwdbtn,False,...,-5.092,0,0.0432,0.07360,0.000000,0.1560,0.500,92.946,185360,4
7zsXy7vlHdItvUSH8EwQss,2022-05-25 08:46:30-04:00,"iOS 15.4.1 (iPhone9,1)",5410,US,Cold Water,Major Lazer,Cold Water,fwdbtn,fwdbtn,False,...,-5.092,0,0.0432,0.07360,0.000000,0.1560,0.500,92.946,185360,4
7zwn1eykZtZ5LODrf7c0tS,2024-04-25 15:20:27-04:00,ios,153000,US,You Get Me So High,The Neighbourhood,Hard To Imagine The Neighbourhood Ever Changing,trackdone,trackdone,True,...,-6.099,0,0.0542,0.18600,0.079100,0.1520,0.387,88.036,153000,4
7zxyRoHDN5U4GhkJ5nzuK8,2022-09-29 09:25:36-04:00,OS X 12.6.0 [x86 8],12123,US,The MegaSIX,SIX,SIX: LIVE ON OPENING NIGHT (Original Broadway ...,trackdone,endplay,False,...,-5.613,1,0.2660,0.01550,0.000006,0.8640,0.309,144.149,168130,4


In [102]:
final_df.to_csv("reid_final_data.csv")

In [88]:
track_aud_feats["mode"].value_counts()

1    3947
0    1602
Name: mode, dtype: int64

In [101]:
final_df["mode"].value_counts()

1    23647
0     8390
Name: mode, dtype: int64